# SITCOM-307
This notebook is to support the boresighting of Elanas SHFWS bench on instrument port 1.
It assumes we are starting at the beginning of the night.
We will choose a target in the east and rising to minimize field rotation

#### Import required libraries

In [1]:
import asyncio
import time
import os
import numpy as np
import logging
import yaml
import jsonschema


from lsst.ts import salobj
from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType
from lsst.ts.idl import enums
from lsst.ts.idl.enums.ScriptQueue import Location, SalIndex

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [2]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

#### Instantiate control classes and ScriptQueue Remote

In [3]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
sq_remote = salobj.Remote(domain,"ScriptQueue", index=SalIndex.AUX_TEL)
await asyncio.gather(atcs.start_task, latiss.start_task, sq_remote.start_task)

[[None, None, None, None, None, None, None], [None, None, None, None], None]

#### Verify that ATAOS corrections are active and dome is following

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
await atcs.enable_dome_following()

## Declare and Slew to CWFS Target
The CWFS and boresight alignment require a ~8th magnitude star, but the SHWFS will use a very bright star (Agena (Hadar), 0.55 mag, technically a double according to simbad?)

In [10]:
cwfs_target = "HIP 67909" # for Elana
cwfs_target = "HD 131258" # for Elana
shwfs_target = "HIP 71683A" #  Rigel Kentaurus (Hadar works too)
shwfs_target = "HR 5459" #  Rigel Kentaurus (Hadar works too)
shwfs_target = "HIP 78265" #  higher altitude target
shwfs_target = "HR 5440" #  higher altitude target

In [ ]:
# Make sure pointing component is using Nasmyth 2
await atcs.rem.atptg.cmd_focusName.set_start(focus=3, timeout=5)

In [15]:
# This could be done via the scriptQueue but this is a one-liner in the TCS
await atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky, rot=0)

In [16]:
await latiss.take_engtest(exptime=5)

[2022052500014]

## Using LATISS, perform a CWFS then a boresight alignment
This is best done using the scriptQueue <br>

### Perform CWFS with LATISS

In [ ]:
is_standard = False  # uses an externalscript
script_path = "auxtel/latiss_cwfs_align.py"
script_config = dict(
    filter= "SDSSr",
    grating= "empty_1",
    reason="LATISS_CWFS_for_SITCOM-307", # replace spaces by underscore due to bug
    program="SITCOM-307"
)
script_description="CWFS for SITCOM-307"

In [ ]:
# One could copy/paste the following into the LOVE GUI if desired.
print(yaml.safe_dump(script_config))

#### Validate the configuration
This will become a one-liner in the future

In [ ]:
sq_remote.evt_configSchema.flush()
await sq_remote.cmd_showSchema.set_start(
    isStandard=is_standard,
    path=script_path,
)
script_schema = await sq_remote.evt_configSchema.next(flush=False, timeout=5)

In [ ]:
config_validator = salobj.DefaultingValidator(schema=yaml.safe_load(script_schema.configSchema))
validation_err = None

try:
    config_validator.validate(script_config)
except jsonschema.ValidationError as e:
    validation_err = e.message

if validation_err is not None:
    raise RuntimeError(validation_err)
else:
    print("Configuration validated")

#### Add the script to the Queue

In [ ]:
ack  = await sq_remote.cmd_add.set_start(
    isStandard=is_standard,
    path=script_path,
    descr=script_description,
    config=yaml.safe_dump(script_config),
    location=Location.LAST,
    logLevel=logging.DEBUG,
)
print(ack)

### Perform Boresighting with LATISS via scriptQueue

In [ ]:
is_standard = False  # uses an externalscript
script_path = "auxtel/latiss_acquire_and_take_sequence.py"
boresight_script_config = dict(
    acq_exposure_time= 2.0,
    acq_filter= "SDSSr",
    acq_grating= "empty_1",
    do_acquire= True,
    do_pointing_model= True,
    max_acq_iter= 3,
    object_name= cwfs_target,
    target_pointing_tolerance= 5.0,
    target_pointing_verification= True,
    reason="align_target_to_boresight", # replace spaces by underscore due to bug
    program="SITCOM-307"
)
script_description="LATISS boresighting for SITCOM-307"

In [ ]:
# One could copy/paste the following into the LOVE GUI if desired.
# print(yaml.safe_dump(script_config))

#### Validate the configuration 

In [ ]:
sq_remote.evt_configSchema.flush()
await sq_remote.cmd_showSchema.set_start(
    isStandard=is_standard,
    path=script_path,
)
script_schema = await sq_remote.evt_configSchema.next(flush=False, timeout=5)

In [ ]:
config_validator = salobj.DefaultingValidator(schema=yaml.safe_load(script_schema.configSchema))
validation_err = None

try:
    config_validator.validate(boresight_script_config)
except jsonschema.ValidationError as e:
    validation_err = e.message

if validation_err is not None:
    raise RuntimeError(validation_err)
else:
    print("Configuration validated")

#### Add the script to the Queue

In [ ]:
ack = await sq_remote.cmd_add.set_start(
    isStandard=is_standard,
    path=script_path,
    descr=script_description,
    config=yaml.safe_dump(boresight_script_config),
    location=Location.LAST,
    logLevel=logging.DEBUG,
)
print(ack)

In [ ]:
# get the current offsets (which will be in RA/Dec)
latiss_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(f'RA offset is: {latiss_offsets.handsetOffsetRA}')
print(f'Dec offset is: {latiss_offsets.handsetOffsetDec}')

#### Absorb the offsets such that the telescope is boresight is now aligned with the camera

In [ ]:
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=1)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=1)

In [ ]:
latiss_offsets2 = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(f'RA offset is: {latiss_offsets2.handsetOffsetRA}')
print(f'Dec offset is: {latiss_offsets2.handsetOffsetDec}')

## Switch Instrument Ports, Will now be using the SHWFS and instrument

In [14]:
await atcs.stop_tracking()

In [ ]:
# Can't use aget or next due to a callback function being used somewhere...
# tmp = await atcs.rem.atmcs.tel_mount_AzEl_Encoders.next(flush=True, timeout=5)
tmp = atcs.rem.atmcs.tel_mount_AzEl_Encoders.get()

In [ ]:
curr_az=tmp.azimuthCalculatedAngle[0]
curr_el=tmp.elevationCalculatedAngle[0]
print(f'Current azimuth is: {curr_az}')
print(f'Current elevation is: {curr_el}')

#### Move telescope to change instrument ports with M3

In [ ]:
await atcs.point_azel(az=curr_az, el=75)

In [ ]:
await atcs.stop_tracking()

#### Change M3 to instrument port 1
Note that the following command won't actually change the rotator due to the mechanical issue. <br>
After this command, the mirror should be moved manually by Alysha

In [ ]:
# Don't use this, the enumeration is incorrect
# await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=enums.ATMCS.M3ExitPort.NASMYTH1)

In [ ]:
# 3 is port 3
# 2 is nasymth 2
# 1 is nasmyth 1
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=2)

In [ ]:
# Make sure pointing component is using Nasmyth 1
await atcs.rem.atptg.cmd_focusName.set_start(focus=2, timeout=5)

### Slew to the SHWFS target
Because Elana will need to manually adjust the acquisition camera in the next steps, we'll fix the rotator for now. <br>
This means that the offset axes determination will have to be done fairly quickly. <br>
After this slew, there are three offsets that need to be accounted for and corrected.
1. The offset between the boresight for LATISS and for INST1.
2. The pointing offset between LATISS and INST1
3. The acquisition camera will have an unknown physical offset. <br>
Also, the camera will be out of focus.

In [8]:
await atcs.slew_object(shwfs_target, rot_type=RotType.Physical, rot=0)

### Elana physically moves the acquisition camera to have the star at the center
At this point, the camera will be at the offset position of 1+2

### Elana manually focuses the acquisition camera, ensuring it stays at the center
acquisition camera is 11.3 mm by 7.1 mm <br>
Pixel size is 5.86 um <br>
Telescope focal length is 21.6m <br>
field is 1.9 by 1.2 arcminutes <br>
pixel scale is 0.001 arcsec/pixel <br>

### Perform X/Y offsets to map the detector axes to sky coordinates

At this point, it is imperative that Elana write down how the star moves on the detector.<br>
Strongly suggest using a sticky note and putting arrows for +X and +Y. <br>
This way she will verbally provide telescope offsets. <br>
If possible it would be useful to rotate the acquisition camera so there is not a rotation to account for <br>

In [ ]:
# Start the instrument rotating
await atcs.slew_object(shwfs_target, rot_type=RotType.PhysicalSky, rot=90)

In [ ]:
# get the current offsets (which will be in RA/Dec)
starting_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(f'RA offset is: {starting_offsets.handsetOffsetRA}')
print(f'Dec offset is: {starting_offsets.handsetOffsetDec}')

In [ ]:
# Start with positive X-offset of 1/4 of the short side of the detector
offset = np.round(1.2*60*0.25)
print(f'offset will be {offset} arcseconds')

In [ ]:
await atcs.offset_xy(x=offset,y=0)

In [ ]:
#now come back to the original
await atcs.offset_xy(x=-offset,y=0)

In [ ]:
#now move in Y
await atcs.offset_xy(x=0,y=offset)

In [ ]:
#now come back to the starting position
await atcs.offset_xy(x=0,y=-offset)

In [ ]:
# Check the offsets are consistent with the originals - note that offsets will be in RA and DEC!
current_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(f'RA offset is: {current_offsets.handsetOffsetRA}')
print(f'Dec offset is: {current_offsets.handsetOffsetDec}')

## Measure Boresight Position
This is performed by taking an image with the rotator moving to form an arc
The center of the arc will be the boresight of the telescope. <br>
The detector will need to be physically moved such that the center of the arc is on the center of the detector <br> 

In [ ]:
# Tell Elana to start exposing before starting the slew!
await atcs.slew_object(shwfs_target, rot_type=RotType.PhysicalSky, rot=90)

### Now offset the telescope to put the star at the center of the detector
Write down the final offsets!!

In [ ]:
await atcs.offset_xy(x=0,y=0)

In [ ]:
final_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(final_offsets)

#### Absorb the offsets such that the telescope is boresight is now aligned with the camera

In [ ]:
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=1)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=1)

In [ ]:
# Not sure these are useful, but they should be zero.
post_absorbed_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(post_absorbed_offsets)

In [4]:
atcs.open_dome_shutter?

Signature: atcs.open_dome_shutter()
Docstring: Task to open dome shutter and return when it is done.
File:      ~/auto-op-env-packages/ts_observatory_control/python/lsst/ts/observatory/control/auxtel/atcs.py
Type:      method
